<div style="background: linear-gradient(to right, #6a11cb, #2575fc); padding: 20px; border-radius: 10px; color: white; text-align: center; margin-bottom: 30px; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
<h1 style="margin: 0; text-shadow: 2px 2px 4px rgba(0,0,0,0.2);">Pricing American Options in Rough Bergomi</h1>
<p style="margin-top: 10px; font-size: 18px; opacity: 0.9;">Using Linear and Deep Signature Methods</p>
</div>

<div style="background-color: #f8f9fa; padding: 15px; border-left: 5px solid #2575fc; margin: 10px 0;">
<p>In this notebook we show how to use the code from <a href="https://github.com/lucapelizzari/Optimal_Stopping_with_signatures/tree/main" style="color: #2575fc; text-decoration: none; font-weight: bold;">this GitHub repository</a>, to compute lower and upper bounds for American options in the rough Bergomi model using different signature methods, see for example Section 4.2 of <a href="https://arxiv.org/abs/2312.03444" style="color: #2575fc; text-decoration: none; font-weight: bold;">this paper</a> for the linear approach, whereas the deep neural network approaches will be discussed in a forthcoming paper.</p>

<p>The repository consists of:</p>

<ul style="list-style-type: none; padding-left: 20px;">
    <li><span style="color: #2575fc; font-weight: bold;">→</span> Simulation packages for fractional Brownian motion, rough Bergomi and rough Heston models</li>
    <li><span style="color: #2575fc; font-weight: bold;">→</span> A modul for signature related computations <code style="background-color: #eef; padding: 2px 4px; border-radius: 3px;">Signature_computer.py</code>, which can compute the signature and log-signature of various lifts related to volatility modelling, with the additional option of adding polynomials of the state-process and/or volatility.</li>
    <li><span style="color: #2575fc; font-weight: bold;">→</span> The main module for the linear signature approaches <code style="background-color: #eef; padding: 2px 4px; border-radius: 3px;">Linear_signature_optimal_stopping.py</code>, which can be used to derive lower and upper bounds to the optimal stopping problem applying the approaches described in <a href="https://arxiv.org/abs/2312.03444" style="color: #2575fc; text-decoration: none; font-weight: bold;">the paper</a>.</li>
    <li><span style="color: #2575fc; font-weight: bold;">→</span> The main module for deep log-signature approaches <code style="background-color: #eef; padding: 2px 4px; border-radius: 3px;">Deep_signature_optimal_stopping.py</code>, which extends the linear approaches by applying deep neural networks on the log-signature. This code is accompanying a working paper on "American option pricing using signatures".</li>
</ul>
</div>

<div style="background-color: #4472C4; color: white; padding: 20px; border-radius: 10px;">
<h1 style="text-align: center; color: white;"> Parameter Estimation From Options Data</h1>
<p style="text-align: center; font-style: italic; font-size: 16px;">A step-by-step guide to estimate rough volatility model parameters from options data</p>
</div>


<p style="text-align: center; margin-top: 15px;">Let's dive in! </p>
</div>

## 1. Importing Required Libraries
<div style="border-left: 5px solid #4472C4; padding-left: 10px;">
First, we'll import the necessary Python libraries for data handling, numerical computations, and file operations.
</div>

In [ ]:
import numpy as np
import pandas as pd
import os
import logging
from pathlib import Path

## 2. Initializing Global Parameters
<div style="background-color: #F2F9FF; padding: 15px; border-radius: 5px; border-left: 5px solid #5B9BD5;">
Here we set up placeholders for our model parameters. These will be populated later in the notebook based on our analysis.

<table style="width:100%; border-collapse: collapse; margin-top:10px;">
  <tr style="background-color: #5B9BD5; color: white;">
    <th style="padding: 8px; text-align: left;">Parameter</th>
    <th style="padding: 8px; text-align: left;">Description</th>
  </tr>
  <tr style="background-color: #EAF2FA;">
    <td style="padding: 8px; font-family: monospace;">N1, N, T</td>
    <td style="padding: 8px;">Time discretization parameters</td>
  </tr>
  <tr>
    <td style="padding: 8px; font-family: monospace;">M, M2</td>
    <td style="padding: 8px;">Monte Carlo simulation parameters</td>
  </tr>
  <tr style="background-color: #EAF2FA;">
    <td style="padding: 8px; font-family: monospace;">eta</td>
    <td style="padding: 8px;">Volatility of volatility parameter</td>
  </tr>
  <tr>
    <td style="padding: 8px; font-family: monospace;">X0</td>
    <td style="padding: 8px;">Initial asset price (forward price)</td>
  </tr>
  <tr style="background-color: #EAF2FA;">
    <td style="padding: 8px; font-family: monospace;">r</td>
    <td style="padding: 8px;">Risk-free interest rate</td>
  </tr>
  <tr>
    <td style="padding: 8px; font-family: monospace;">rho</td>
    <td style="padding: 8px;">Correlation between asset returns and volatility</td>
  </tr>
  <tr style="background-color: #EAF2FA;">
    <td style="padding: 8px; font-family: monospace;">xi</td>
    <td style="padding: 8px;">Initial variance value</td>
  </tr>
  <tr>
    <td style="padding: 8px; font-family: monospace;">strike</td>
    <td style="padding: 8px;">Option strike price</td>
  </tr>
  <tr style="background-color: #EAF2FA;">
    <td style="padding: 8px; font-family: monospace;">K</td>
    <td style="padding: 8px;">Model calibration parameter</td>
  </tr>
</table>
</div>

### 2.1 The Payoff Function
<div style="background-color: #F8F8F8; padding: 12px; border-left: 4px solid #70AD47; margin: 10px 0;">
For put options, the payoff function determines the final value of the option at expiration. It returns the maximum of (strike price - asset price) or zero.

<div style="text-align: center; margin-top: 15px;">
$\text{Payoff} = \max(K - S_T, 0)$
</div>

<p style="text-align: center; font-style: italic; margin-top: 5px; color: #555;">where $K$ is the strike price and $S_T$ is the asset price at expiration</p>
</div>

In [ ]:
# Global option model parameters (pre-initialized)
N1 = 0
N = 0
T = 0
T_years = 0.0
M = 0
M2 = 0
eta = 0.0
X0 = 0.0
r = 0.0
rho = 0.0
xi = 0.0
strike = 0.0
K = 0
Hurst_list = []

def phi(x):
    return np.maximum(strike - x, 0)


Paths/ Loading_Cleaning the data:

## 3. Setting Up File Paths / Data Loading Function
<div style="background-color: #FFF4E1; padding: 10px; border-radius: 5px; border-left: 5px solid #FF9800;">
Now we define the file paths for our data. We use Path from the pathlib library to ensure cross-platform compatibility.
</div>
<div style="background-color:#F3E5F5; width:95%; padding:15px; border-radius:10px; border-left:5px solid #7B1FA2">
This function handles loading the price dataset. It checks if a cached version exists (for faster loading) and otherwise loads from CSV.
</div>

In [ ]:
# Logging setup
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Paths
repo_root = os.path.abspath('..')
notebook_dir = Path(repo_root) / 'XGboost_Roshan' / 'src'
DATA_PICKLE = notebook_dir / 'data' / 'dataset1.pkl'
RETURNS_PICKLE = notebook_dir / 'data' / 'dataset3.pkl'

def load_and_clean_data(pickle_path):
    if os.path.exists(pickle_path):
        logger.info(f"Loading data from {pickle_path}")
        df = pd.read_pickle(pickle_path)
    else:
        logger.warning(f"Pickle file not found at {pickle_path}")
        os.makedirs(pickle_path.parent, exist_ok=True)
        csv_path = pickle_path.with_suffix('.csv')
        if os.path.exists(csv_path):
            logger.info(f"Found CSV at {csv_path}. Converting to pickle.")
            df = pd.read_csv(csv_path, parse_dates=['date'], dayfirst=False)
            df.to_pickle(pickle_path)
            logger.info(f"Converted and saved as pickle: {pickle_path}")
        else:
            logger.warning(f"CSV file also not found at {csv_path}")
            return None

    if 'secid' in df.columns or 'index_flag' in df.columns:
        df = df.drop(columns=['secid', 'index_flag'], errors='ignore')
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])

    logger.info(f"Loaded {df.shape[0]} rows and {df.shape[1]} columns from {pickle_path.name}")
    return df

def select_data_by_date_and_days(df, target_date=None, days_length=None):
    filtered_df = df.copy()
    if target_date:
        filtered_df = filtered_df[filtered_df['date'] == pd.to_datetime(target_date)]
    if days_length is not None:
        filtered_df = filtered_df[filtered_df['days'] == days_length]
    return filtered_df


def extract_model_inputs(option_df):
    put_df = option_df[option_df['cp_flag'] == 'P']
    if put_df.empty:
        return None
    row = put_df.iloc[0]
    return {
        "ticker": row['ticker'],
        "date": row['date'],
        "T_years": row['days'] / 252,
        "strike": row['strike_price'],
        "X0 (forward)": row['forward_price'],
        "market_premium": row['premium']
    }

## 4. Parameter Estimation Functions

> Now we'll define functions to estimate two critical model parameters for rough volatility models:

---

### ρ (rho)
- **Meaning**: Correlation between returns and volatility  
- **Formula**:  
  $$
  \rho = \text{Corr}(r_t, \Delta \sigma_t)
  $$

---

### η (eta)
- **Meaning**: Volatility of volatility  
- **Formula**:  
  $$
  \eta = \frac{\text{std}(\Delta \log \sigma_t)}{(\Delta t)^H}
  $$

These parameters are crucial for capturing the dynamics of financial markets accurately.  
We also use a **rolling Hurst exponent \( H \)** in the denominator of η to reflect local path roughness. Which we got from <code style="background-color: #eef; padding: 2px 4px; border-radius: 3px;">train_hurst.py</code>

In [ ]:
def estimate_rho(returns_df, options_df, target_ticker, target_date, days_length):
    ret_df = returns_df[(returns_df['ticker'] == target_ticker) & (returns_df['date'] <= target_date)].copy()
    opt_df = options_df[(options_df['ticker'] == target_ticker) & 
                        (options_df['cp_flag'] == 'P') & 
                        (options_df['days'] == days_length) & 
                        (options_df['date'] <= target_date)].copy()

    ret_df = ret_df.sort_values('date')
    opt_df = opt_df.sort_values('date').drop_duplicates(subset=['date'])

    merged = pd.merge(ret_df, opt_df[['date', 'impl_volatility']], on='date', how='inner')
    merged['vol_change'] = merged['impl_volatility'].diff()
    merged = merged.dropna(subset=['return', 'vol_change'])

    if len(merged) < 2:
        return None
    return merged['return'].corr(merged['vol_change'])

def compute_rolling_hurst(returns: np.ndarray, power: int) -> np.ndarray:
    if not isinstance(returns, np.ndarray):
        returns = np.array(returns)
    n = 2**power
    if len(returns) < n:
        raise ValueError(f"Need at least {n} data points for power={power}")
    hursts = []
    exponents = np.arange(2, power+1)
    for t in range(n, len(returns) + 1):
        window = returns[t-n:t]
        rs_log = []
        for exp in exponents:
            m = 2**exp
            s = n // m
            segments = window.reshape(s, m)
            dev = np.cumsum(segments - segments.mean(axis=1, keepdims=True), axis=1)
            R = dev.max(axis=1) - dev.min(axis=1)
            S = segments.std(axis=1)
            rs = np.where(S != 0, R/S, 0)
            rs_log.append(np.log2(rs.mean()))
        hursts.append(np.polyfit(exponents, rs_log, 1)[0])
    return np.array(hursts)

def estimate_eta_and_xi(option_df, hurst):
    option_df = option_df.sort_values('date').drop_duplicates(subset=['date'])
    option_df['log_vol'] = np.log(option_df['impl_volatility'])
    option_df['log_vol_diff'] = option_df['log_vol'].diff()
    std_log_vol_diff = option_df['log_vol_diff'].dropna().std()
    delta_t = 1 / 252
    eta = std_log_vol_diff / (delta_t ** hurst)
    xi = option_df['impl_volatility'].iloc[0] ** 2 if not option_df['impl_volatility'].isnull().all() else 0.0
    return eta, xi


## 7. Main Analysis Process
<div style="background-color: #E8EAF6; padding: 15px; border-radius: 8px; border-left: 5px solid #3F51B5;">
Now let's define our main analysis function that brings everything together. This function will:

<ol style="color: #333; margin-top: 10px;">
  <li>Filter the option data by ticker, date, and days to maturity</li>
  <li>Extract model inputs from the filtered put options</li>
  <li>Estimate the correlation (ρ) between returns and volatility</li>
  <li>Compute the Hurst exponent (H) for the roughness of volatility</li>
  <li>Estimate volatility parameters (η and ξ)</li>
  <li>Set up all required model configuration parameters</li>
</ol>



In [ ]:
if __name__ == "__main__":
    df = load_and_clean_data(DATA_PICKLE)
    returns_df = load_and_clean_data(RETURNS_PICKLE)

    if df is not None and returns_df is not None:
        date_input = input("Enter date (YYYY-MM-DD) [default: 2023-08-31]: ").strip()
        target_date = pd.to_datetime(date_input if date_input else "2023-08-31")

        days_input = input("Enter option length in days [default: 10]: ").strip()
        try:
            days_length = int(days_input) if days_input else 10
        except ValueError:
            logger.warning("Invalid input for days. Defaulting to 10.")
            days_length = 10

        ticker_input = input("Enter ticker [default: AAPL]: ").strip().upper()
        ticker = ticker_input if ticker_input else "AAPL"

        filtered_data = select_data_by_date_and_days(df, target_date, days_length)
        ticker_data = filtered_data[filtered_data['ticker'] == ticker]

        if len(ticker_data) > 0:
            print(f"\nFiltered data for {ticker} on {target_date.date()} with {days_length} days:")
            print(ticker_data)
            model_inputs = extract_model_inputs(ticker_data)
            if model_inputs:
                print("\n Extracted Model Inputs (PUTS):")
                for k, v in model_inputs.items():
                    print(f"{k}: {v}")

                rho_est = round(estimate_rho(returns_df, df, ticker, target_date, days_length), 2)
                print(f"Estimated rho: {rho_est}" if rho_est is not None else "Rho estimation failed.")
                rho = rho_est if rho_est is not None else -0.9

                rets = returns_df[(returns_df['ticker'] == ticker) & (returns_df['date'] <= target_date)]
                rets = rets.sort_values('date')['return'].values
                hurst_values = compute_rolling_hurst(rets, power=5)
                hurst_dates = returns_df[(returns_df['ticker'] == ticker) & (returns_df['date'] <= target_date)].sort_values('date').iloc[2**5 - 1:]['date'].values
                hurst_df = pd.DataFrame({'date': hurst_dates, 'H': hurst_values})
                H_df = hurst_df[hurst_df['date'] <= target_date]
                latest_H = H_df.iloc[-1]['H'] if not H_df.empty else 0.1
                print(f"Using Hurst: {round(latest_H, 3)}")

                iv_path_df = df[
                    (df['ticker'] == ticker) &
                    (df['cp_flag'] == 'P') &
                    (df['days'] == days_length) &
                    (df['date'] <= target_date)
                ].sort_values('date')

                eta, xi = estimate_eta_and_xi(iv_path_df, hurst=latest_H)
                eta = round(eta, 3)
                xi = round(xi, 5)

                N = 252
                M = 2**13
                M2 = 2**13
                
                r = 0.05
                K = 2

                T = int(model_inputs["T_years"] * 252)
                N1 = T
                premium = model_inputs["market_premium"]
                T_years = model_inputs["T_years"]
                X0 = model_inputs["X0 (forwa" \
                "rd)"]
                strike = model_inputs["strike"]

                print("\n✅ Global parameters set:")
                print(f"N1 = {N1}, N = {N}, T = {T}, T_years = {T_years:.5f}")
                print(f"M = {M}, M2 = {M2}, eta = {eta}, X0 = {X0}, strike = {strike}")
                print(f"r = {r}, rho = {rho}, xi = {xi}, K = {K}")
            else:
                logger.warning("No put option found for this filter.")
        else:
            logger.warning("No data matches your filter criteria.")

In [ ]:
print(f"n1 = {N1}, n = {N}, T = {T}, T_years = {T_years:.5f}, M = {M}, M2 = {M2}, eta = {eta:}, X0 = {X0:.5f}, strike = {strike:.5f}, r = {r:.5f}, rho = {rho:.5f}, xi = {xi:.5f}, K = {K}")

In [ ]:
# Fix for TensorFlow import issues and environment compatibility
# Set correct Python path to find modules
import sys
import os

# Add the root directory and subdirectories to Python path
repo_root = os.path.abspath('..')
sys.path.append(repo_root)
sys.path.append(os.path.join(repo_root, "Linear signature optimal stopping"))
sys.path.append(os.path.join(repo_root, "Non linear signature optimal stopping"))

# Suppress TensorFlow warnings
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

## Now I will Estimating my Hurst Parameter:

# <span style="color:#1E88E5">XGBoost for Hurst Exponent Forecasting (over multiple days) </span> - Step by Step Tutorial

This notebook provides a detailed walkthrough of `train_hurst.py`, a script that computes the rolling Hurst exponent of a time series and trains/evaluates an XGBoost model to forecast its future values.

## <span style="color:#43A047">What is the Hurst Exponent?</span>

<div style="background-color:#EFF8FB; max-width: 700px; padding:15px; border-radius:10px; border-left:5px solid #4682B4">
The Hurst exponent is a measure used in time series analysis that quantifies the long-term memory of a series. It helps determine if a time series is:
<ul>
  <li><span style="color:#F44336"><b>H < 0.5</b></span>: Anti-persistent (mean-reverting)</li>
  <li><span style="color:#9E9E9E"><b>H = 0.5</b></span>: Random walk (no memory)</li>
  <li><span style="color:#4CAF50"><b>H > 0.5</b></span>: Trend-reinforcing (persistent)</li>
</ul>
</div>

## <span style="color:#43A047">What This Notebook Covers</span>

<div style="display: flex; flex-wrap: wrap; gap: 10px; margin-top: 10px">
  <div style="flex: 1; min-width: 100px; background-color:#E3F2FD; padding:10px; border-radius:5px">📚 <b>1.</b> Importing libraries</div>
  <div style="flex: 1; min-width: 150px; background-color:#E8F5E9; padding:10px; border-radius:5px">📈 <b>2.</b> Computing rolling Hurst</div>
  <div style="flex: 1; min-width: 150px; background-color:#FFF8E1; padding:10px; border-radius:5px">💾 <b>3.</b> Loading financial data</div>
  <div style="flex: 1; min-width: 150px; background-color:#F3E5F5; padding:10px; border-radius:5px">⚙️ <b>4.</b> Creating lagged features</div>
  <div style="flex: 1; min-width: 150px; background-color:#E0F7FA; padding:10px; border-radius:5px">🧠 <b>5.</b> Training XGBoost model</div>
  <div style="flex: 1; min-width: 150px; background-color:#FFEBEE; padding:10px; border-radius:5px">🔍 <b>6.</b> Evaluating performance</div>
  <div style="flex: 1; min-width: 150px; background-color:#F1F8E9; padding:10px; border-radius:5px">🔮 <b>7.</b> Forecasting future values</div>
  <div style="flex: 1; min-width: 150px; background-color:#E8EAF6; padding:10px; border-radius:5px">🖥️ <b>8.</b> Interactive menu system</div>
</div>

Let's get started!

In [ ]:
# Import necessary libraries for numerical operations, data manipulation, and visualization
import numpy as np                # For numerical computations (arrays, math functions)
import pandas as pd               # For data manipulation with DataFrames
import matplotlib.pyplot as plt   # For creating plots and visualizations
from pathlib import Path          # For handling file paths in a platform-independent way
from sklearn.metrics import mean_squared_error, mean_absolute_error  # For model evaluation metrics
import logging                    # For logging status and debug messages
import joblib                     # For saving/loading Python objects (like our trained models)
from xgboost import XGBRegressor  # XGBoost regression model implementation

# Set up basic configuration
# How many days ahead to predict (forecast horizon)
HORIZON = T

# Configure logging to show informational messages with timestamps
logging.basicConfig(
    level=logging.INFO,  # Show INFO level messages and above
    format='%(asctime)s - %(levelname)s - %(message)s'  # Include timestamp and message level
)
logger = logging.getLogger(__name__)

# Create directory for saving trained models
MODELS_DIR = notebook_dir / 'models'
MODELS_DIR.mkdir(parents=True, exist_ok=True)  # Create directory if it doesn't exist

print(f"Models will be stored in: {MODELS_DIR}")

## <span style="color:#FF8F00">2. Computing the Rolling Hurst Exponent</span>
<hr style="height:1px;border:none;background-color:#FF8F00">

This function calculates the Hurst exponent using Rescaled Range (R/S) analysis over a rolling window. The Hurst exponent measures the long-term memory or persistence of a time series.

<div style="background-color:#FFF8E1; max-width: 700px; padding:15px; border-radius:10px; border-left:5px solid #FFB300">
<span style="font-size:1.1em">🔑 <b>Key Concepts:</b></span>
<ul>
  <li><b>Window size</b>: 2<sup>power</sup> determines how many data points we use for each calculation</li>
  <li><b>R/S Analysis</b>: Calculates the ratio of the range to the standard deviation at different time scales</li>
  <li><b>Slope of log-log plot</b>: The slope of the line relating log(R/S) to log(time scale) gives us the Hurst exponent</li>
</ul>
</div>

In [ ]:
def compute_rolling_hurst(returns: np.ndarray, power: int) -> np.ndarray:
    """
    Calculate the Hurst exponent using R/S analysis over a rolling window of size 2^power.
    
    Parameters:
    -----------
    returns : np.ndarray
        Time series data (e.g., asset returns) to analyze
    power : int
        Power of 2 to determine window size (window size = 2^power)
        
    Returns:
    --------
    np.ndarray
        Array of Hurst exponents calculated over rolling windows
    """
    # Convert input to numpy array if it's not already
    if not isinstance(returns, np.ndarray):
        returns = np.array(returns)
        
    # Validate the power parameter
    if not isinstance(power, int) or power < 1:
        raise ValueError("power must be a positive integer")
        
    # Calculate window length as 2^power
    n = 2**power
    
    # Check if we have enough data
    if len(returns) < n:
        raise ValueError(f"Need at least {n} data points for power={power}")
        
    # Initialize list to store Hurst exponents
    hursts = []
    
    # Define the range of exponents for multi-scale analysis
    # (from 2 to power, these will be our different time scales)
    exponents = np.arange(2, power+1)
    
    # Roll the window through the data
    for t in range(n, len(returns) + 1):
        # Get current window of data
        window = returns[t-n:t]
        
        # Store log values of R/S at different scales
        rs_log = []
        
        # Calculate R/S at different time scales
        for exp in exponents:
            # Size of each segment at this scale
            m = 2**exp
            
            # Number of segments
            s = n // m
            
            # Reshape data into segments
            segments = window.reshape(s, m)
            
            # Calculate cumulative deviation from mean for each segment
            dev = np.cumsum(
                segments - segments.mean(axis=1, keepdims=True),
                axis=1
            )
            
            # Range (R) is max deviation minus min deviation
            R = dev.max(axis=1) - dev.min(axis=1)
            
            # Standard deviation (S) of each segment
            S = segments.std(axis=1)
            
            # Calculate R/S ratio (avoid division by zero)
            rs = np.where(S != 0, R/S, 0)
            
            # Store log2 of mean R/S value
            rs_log.append(np.log2(rs.mean()))
        
        # Fit a line to log(R/S) vs log(time scale)
        # The slope of this line is the Hurst exponent
        hursts.append(np.polyfit(exponents, rs_log, 1)[0])
        
    return np.array(hursts)

In [ ]:
def load_data() -> pd.DataFrame:
    """
    Load the price dataset from CSV or cache, sort by date,
    and return a pandas DataFrame.
    
    Returns:
    --------
    pd.DataFrame
        DataFrame containing price data sorted by date
    """
    # First, check if a cached (pickled) version of the data exists
    if RETURNS_PICKLE.exists():
        # Load from the pickle file (much faster than CSV)
        df = pd.read_pickle(RETURNS_PICKLE)
        logger.info(f"Loaded data from cache: {RETURNS_PICKLE}")
    else:
        # Sort the data by date
        df.sort_values('date', inplace=True)
        
        # Cache the data for faster future loading
        df.to_pickle(RETURNS_PICKLE)
        logger.info(f"Saved data to cache: {RETURNS_PICKLE}")
        
    return df

## <span style="color:#26A69A">4. Creating Lagged Features for Time Series Forecasting</span>
<hr style="height:1px;border:none;background-color:#26A69A">

This function creates lagged features for time series forecasting. Lagged features are simply past values of the time series that are used to predict future values.

<div style="display: flex; justify-content: center; margin: 20px 0;">
  <div style="background-color:#E0F2F1; padding:15px; border-radius:10px; width:90%">
    <span style="color:#00695C; font-weight:bold">Key Concepts:</span>
    <table style="width:100%; border-collapse: collapse; margin-top: 10px">
      <tr style="background-color:#B2DFDB">
        <th style="padding:8px; text-align:left">Concept</th>
        <th style="padding:8px; text-align:left">Description</th>
      </tr>
      <tr>
        <td style="padding:8px; border-bottom:1px solid #ddd"><b>Lag</b></td>
        <td style="padding:8px; border-bottom:1px solid #ddd">Using past values (t-1, t-2, ..., t-k) to predict the current value (t)</td>
      </tr>
      <tr>
        <td style="padding:8px; border-bottom:1px solid #ddd"><b>Feature Matrix X</b></td>
        <td style="padding:8px; border-bottom:1px solid #ddd">Each row contains k consecutive past values</td>
      </tr>
      <tr>
        <td style="padding:8px"><b>Target Vector y</b></td>
        <td style="padding:8px">Contains the value we want to predict (the value at time t)</td>
      </tr>
    </table>
  </div>
</div>

In [ ]:
def make_lagged_features(series: pd.Series, k: int):
    """
    Generate lagged features matrix X and target vector y from a time series.
    
    Parameters:
    -----------
    series : pd.Series
        Time series data to create lagged features from
    k : int
        Number of lagged values to use as features
        
    Returns:
    --------
    (np.ndarray, np.ndarray)
        X: Feature matrix where each row contains k consecutive past values
        y: Target vector containing the values to predict
    """
    # Convert series to a numpy array for faster processing
    vals = series.values
    
    # Initialize empty lists for features and targets
    X, y = [], []
    
    # For each point in the time series (starting from position k)
    for i in range(k, len(vals)):
        # Add the k previous values as features
        X.append(vals[i-k:i])
        # Add the current value as the target
        y.append(vals[i])
    
    # Convert lists to numpy arrays and return
    return np.array(X), np.array(y)

In [ ]:
def train_xgb_only(hurst_series: pd.Series, train_frac: float, k: int, xgb_params: dict):
    """
    Train a pure XGBoost model on lagged Hurst series.
    
    Parameters:
    -----------
    hurst_series : pd.Series
        Series of Hurst exponent values
    train_frac : float
        Fraction of data to use for training (0 to 1)
    k : int
        Number of lagged values to use as features
    xgb_params : dict
        Parameters for the XGBoost model
        
    Returns:
    --------
    XGBRegressor
        Trained XGBoost model
    """
    # Split data into training and test sets based on time
    # (first train_frac portion for training)
    split = int(len(hurst_series) * train_frac)
    train = hurst_series.iloc[:split]
    
    # Create lagged features for training data
    X_tr, y_tr = make_lagged_features(train, k)
    
    # Initialize and train the XGBoost model
    xgb_only = XGBRegressor(**xgb_params)
    xgb_only.fit(X_tr, y_tr)
    
    # Save the trained model to disk for later use
    joblib.dump(xgb_only, MODELS_DIR / 'xgb_only.pkl')
    logger.info("Pure XGBoost model saved to %s", MODELS_DIR)
    
    return xgb_only

In [ ]:
def evaluate_xgb_only(hurst_series: pd.Series, xgb_only, k: int, test_frac: float):
    """
    Evaluate pure XGBoost model on test split and plot results.
    
    Parameters:
    -----------
    hurst_series : pd.Series
        Series of Hurst exponent values
    xgb_only : XGBRegressor
        Trained XGBoost model
    k : int
        Number of lagged values used as features
    test_frac : float
        Fraction of data to use for testing (0 to 1)
        
    Returns:
    --------
    (float, float)
        MSE and MAE on the test set
    """
    # Split data into training and test sets based on time
    # (last test_frac portion for testing)
    split = int(len(hurst_series) * (1 - test_frac))
    test = hurst_series.iloc[split:]
    
    # Create lagged features for test data
    X_te, y_te = make_lagged_features(test, k)
    
    # Use the model to make predictions on test data
    y_pred = xgb_only.predict(X_te)
    
    # Calculate error metrics
    mse = mean_squared_error(y_te, y_pred)
    mae = mean_absolute_error(y_te, y_pred)
    
    # Log the results
    logger.info("XGBoost-only results – MSE: %.4f, MAE: %.4f", mse, mae)
    
    # Plot actual vs predicted values
    plt.figure(figsize=(12,6))
    
    # Get dates for the test set (need to skip the first k points because of lagging)
    dates_te = test.index[k:]
    
    # Plot actual values
    plt.plot(dates_te, y_te, 'k-', label='Actual Hurst')
    
    # Plot predicted values
    plt.plot(dates_te, y_pred, 'g--', label='XGB-only Forecast')
    
    # Add title and labels
    plt.title('XGBoost-only: Actual vs Forecast Hurst Exponent')
    plt.xlabel('Date')
    plt.ylabel('Hurst Exponent')
    
    # Add legend, grid, and format plot
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    return mse, mae

In [ ]:
def forecast_xgb_only(hurst_series: pd.Series, xgb_only, periods: int, k: int):
    """
    Generate future forecasts using pure XGBoost model recursively.
    
    Parameters:
    -----------
    hurst_series : pd.Series
        Series of Hurst exponent values
    xgb_only : XGBRegressor
        Trained XGBoost model
    periods : int
        Number of future periods to forecast
    k : int
        Number of lagged values used as features
        
    Returns:
    --------
    pd.DataFrame
        DataFrame with dates and forecasted values
    """
    # Create a copy of historical values to avoid modifying the original
    hist = list(hurst_series.values)
    
    # Initialize list to store predictions
    preds = []
    
    # Generate predictions for each future period
    for _ in range(periods):
        # If we don't have enough historical values yet, default to 0
        if len(hist) < k:
            p = 0
        else:
            # Use the model to predict the next value
            p = xgb_only.predict(np.array(hist[-k:]).reshape(1, -1))[0]
        
        # Store the prediction
        preds.append(p)
        
        # Add the prediction to historical values for recursive forecasting
        hist.append(p)
    
    # Generate future dates starting from the day after the last historical date
    # 'B' frequency means business days (Monday to Friday)
    dates = pd.date_range(
        hurst_series.index[-1] + pd.Timedelta(days=1), 
        periods=periods, 
        freq='B'
    )
    
    # Return DataFrame with dates and forecasts
    return pd.DataFrame({'ds': dates, 'yhat': np.array(preds)})

In [ ]:
def main_hurst():
    """
    Main entry point: show menu, get user choice, compute Hurst series,
    and train/evaluate/forecast the XGBoost model as requested.
    """

    # Log start of execution
    logger.info("Starting main execution...")
    # for the sake of this example, we will use a fixed mode
    power = 5
    train_frac, test_frac = 0.8, 0.2
    k = 5
    xgb_params = {'n_estimators': 100, 'learning_rate': 0.1}
    periods = HORIZON

    # Load data and filter by ticker symbol
    df_all = load_data()
    df_t = df_all[df_all['ticker'] == ticker].sort_values('date')
    returns = df_t['return'].values

    # Compute rolling Hurst exponent series
    hurst_vals = compute_rolling_hurst(returns, power)
    dates = df_t['date']
    start_index = 2**power - 1
    hurst_series = pd.Series(hurst_vals, index=dates[start_index:])

    #it will first train the model, then evaluate, and finally forcase
    train_xgb_only(hurst_series, train_frac, k, xgb_params)
    xgb_only = joblib.load(MODELS_DIR / 'xgb_only.pkl')
    evaluate_xgb_only(hurst_series, xgb_only, k, test_frac)
    xgb_only = joblib.load(MODELS_DIR / 'xgb_only.pkl')
    df_fc = forecast_xgb_only(hurst_series, xgb_only, periods, k)
    print("List of hurst parameters:", list(df_fc['yhat']))
    logger.info("XGBoost-only forecast:\n%s", df_fc)
        
        
    # Plot historical and forecasted values
    plt.figure(figsize=(12, 6))
    plt.plot(hurst_series.index, hurst_series.values, 'k-', label='Historical Hurst')
    plt.plot(df_fc['ds'], df_fc['yhat'], 'g--', label='Forecast')
    plt.title(f'{periods}-Day Hurst Forecast for {ticker}')
    plt.xlabel('Date')
    plt.ylabel('Hurst Exponent')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    return list(list(df_fc['yhat']))

In [ ]:
Hurst_list = main_hurst()

<div style="background: linear-gradient(to right, #2575fc, #6a11cb); padding: 10px; border-radius: 8px; margin: 15px 0;">
<h2 style="color: white; margin: 0; padding: 5px;">American Put options in the rough Bergomi model</h2>
</div>

<div style="background-color: #f0f4f8; padding: 15px; border-radius: 5px; border-left: 5px solid #2575fc;">
Recall the price and volatility dynamics of the latter are given by 
<div style="text-align: center; margin: 10px 0;">
\begin{align*}
dX_t &= rX_tdt+X_tv_t \left (\rho dW_r+\sqrt{1-\rho^2}dB_t\right ), \\ 
v_t & =\xi_0\mathcal{E}\left (\eta \int_0^t(t-s)^{H-\frac{1}{2}}dW_s \right )
\end{align*}
</div>
and pricing an American Put-option can be formulated as optimal stopping problem 
<div style="text-align: center; margin: 10px 0;">
$$y_0=\sup_{\tau \in \mathcal{S}_0}\mathbb{E}[e^{-r\tau}\left (K-X_{\tau}\right )^{+}]$$
</div>
for some strike $K$. In this notebook we consider the following choice of paramteres 
<div style="text-align: center; margin: 10px 0; font-weight: bold; color: #2575fc;">
$$ H=0.07,X_0 = 100, r=0.05, \eta = 1.9, \rho = -0.9, \xi_0= 0.09, K = 110.$$
</div>
</div>

<div style="background: linear-gradient(to right, #2575fc, #6a11cb); padding: 10px; border-radius: 8px; margin: 15px 0;">
<h2 style="color: white; margin: 0; padding: 5px;">Step 1: Simulation rough Bergomi model</h2>
</div>

In [ ]:
# Adjust path to include repository root
import sys
import os
sys.path.append(os.path.abspath('..'))
from rBergomi_simulation import SimulationofrBergomi
from dynamic_hurst_rbergomi import SimulationWithDynamicHurst 

In [ ]:
def generate_data_dynamic_hurst(M, N, T_years, phi, rho, K, X0, H_series, xi, eta, r):
    """
    Generate simulation data with time-varying Hurst parameter.
    
    Parameters:
    -----------
    Similar to SimulationWithDynamicHurst, but returns formatted data ready for model training.
    """
    X, V, I, dI, dW1, dW2, dB, Y = SimulationWithDynamicHurst(
        M, N, T_years, phi, rho, K, X0, H_series, xi, eta, r
    )

    # Calculate Payoff
    Payoff = phi(X)

    # Stack state and volatility into features for signature
    MM = np.stack([X, V], axis=-1)

    return X, V, Payoff, dW1, I, MM

In [ ]:
def generate_data(M, N, T_years, phi, rho, K, X0, H, xi, eta, r):
    X, V, I, dI, dW1, dW2, dB, Y = SimulationofrBergomi(M, N, T_years, phi, rho, K, X0, H, xi, eta, r)

    # Calculate Payoff
    Payoff = phi(X)

    # Stack state and volatility into features for signature
    MM = np.stack([X, V], axis=-1)

    return X, V, Payoff, dW1, I, MM



In [ ]:
#defining hurst parameter... last official step to figure out
if Hurst_list is not None:
    H = Hurst_list
    print(f"Using Hurst: {H}")
    S_training, V_training, Payoff_training, dW_training, I_training, MM_training = generate_data_dynamic_hurst(
    M, N, T_years, phi, rho, K, X0, H, xi, eta, r
    )

    S_testing, V_testing, Payoff_testing, dW_testing, I_testing, MM_testing = generate_data_dynamic_hurst(
        M2, N, T_years, phi, rho, K, X0, H, xi, eta, r
    )

else:
    H = 0.07  # Hurst parameter
    # Use K in your function call, not strike
    S_training, V_training, Payoff_training, dW_training, I_training, MM_training = generate_data(M, N, T_years, phi, rho, K, X0, H, xi, eta, r)
    S_testing, V_testing, Payoff_testing, dW_testing, I_testing, MM_testing = generate_data(M2, N, T_years, phi, rho, K, X0, H, xi, eta, r)


print(f"Hurst parameter: {H}")

<div style="background-color: #f0f4f8; padding: 15px; border-radius: 5px; border-left: 5px solid #2575fc;">
Next we define a function generating rough Bergomi prices, volatilies and the corresponding Brownian motion and payoff process. Then we generate training and testing data.
</div>

In [ ]:
print("Show Head of the training data")
print("S_training", S_training[:5])
print("V_training", V_training[:5])
print("Payoff_training", Payoff_training[:5])
print("dW_training", dW_training[:5])
print("I_training", I_training[:5])
print("MM_training", MM_training[:5])


In [ ]:
#compute the volatility processes
vol_training = np.sqrt(V_training)
vol_testing = np.sqrt(V_testing)

<div style="background: linear-gradient(to right, #2575fc, #6a11cb); padding: 10px; border-radius: 8px; margin: 15px 0;">
<h2 style="color: white; margin: 0; padding: 5px;">Step 2: Signature computations</h2>
</div>

<div style="background-color: #f0f4f8; padding: 15px; border-radius: 5px;">
We will make us uf the <a href="https://pypi.org/project/iisignature/" style="color: #2575fc; text-decoration: none; font-weight: bold;">iisignature package</a> to compute the signature, and it can be installed using pip:
</div>

<div style="background-color: #f0f4f8; padding: 15px; border-radius: 5px;">
We import our signature computation module, which can compute various signature and log signature lift related to the generated data
</div>

In [ ]:
from Signature_computer import SignatureComputer

<div style="background-color: #f0f4f8; padding: 15px; border-radius: 5px; border-left: 5px solid #2575fc;">
<p>Next we initialize the <code style="background-color: #eef; padding: 2px 4px; border-radius: 3px;">SignatureComputer</code>, which allows to choose from the linear and the log signature, and various choices of signature lifts. Here are some examples:</p>
<div style="text-align: center; background-color: #f8f9fa; padding: 10px; margin: 10px 0; border-radius: 5px;">
$$ t\mapsto \mathrm{Sig}(A_t,X_t),t\mapsto \mathrm{Sig}(A_t,\phi(X)_t),t\mapsto \mathrm{Sig}(A_t,X_t,X_{t-\epsilon}),t\mapsto \mathrm{Sig}(A_t,X_t,\phi(X_t)),t\mapsto \mathrm{Sig}(A_t,v_t),$$
</div>
<p>where $t\mapsto A_t$ is a monoton path and in our examples we choose between:</p>
<div style="text-align: center; font-weight: bold; color: #2575fc; margin: 10px 0;">
$$A_t=t, \quad  A_t = \langle X\rangle_t.$$
</div>
<p>Additonally we can add Laguerre polynomials of $X$ or $(X,v)$ to the signature, see the module for all the details.</p>
<p>In this example we choose the basis $(\mathrm{Sig}(t,v_t),p_i(X_t))$, which proves to be a solid choice for rough volatility models. We choose both the polynomial and signature degree to be $3$ for this example. (To improve result one should higher truncations levels ($4-5$) for the signature, but to keep the complexity reasonable here we choose level $3$ signatures.)</p>
</div>

In [ ]:
#initialize signature computer
sig_computer = SignatureComputer(T, N, 3, "linear", signature_lift="polynomial-vol", poly_degree=3)

In [ ]:
#Compute the signature for training and test data
tt = np.linspace(0,T,N+1)
A_training = np.zeros((M, N+1)) #time-augmentation
A_testing = np.zeros((M2, N+1))
A_training[:, 1:] = A_testing[:, 1:] = tt[1:]
signatures_training = sig_computer.compute_signature(
    S_training, vol_training, A_training, Payoff_training,
    dW_training, I_training, MM_training
)
signatures_testing = sig_computer.compute_signature(
    S_testing, vol_testing, A_testing, Payoff_testing,
    dW_testing, I_testing, MM_testing
)

<div style="background: linear-gradient(to right, #2575fc, #6a11cb); padding: 10px; border-radius: 8px; margin: 15px 0;">
<h2 style="color: white; margin: 0; padding: 5px;">Step 3: Compute pricing intervals with linear signatures</h2>
</div>

<div style="background-color: #f0f4f8; padding: 15px; border-radius: 5px; border-left: 5px solid #2575fc;">
<p>We can now import the linear primal and dual pricers, which compute true lower and upper bounds.</p>
<ul style="list-style-type: none; padding-left: 20px;">
    <li><span style="color: #2575fc; font-weight: bold;">→</span> The <code style="background-color: #eef; padding: 2px 4px; border-radius: 3px;">LinearLongstaffSchwartzPricer</code> uses the signature of the training data to recursively approximate continuation values in the spirit of the Longstaff-Schwartz algorithm (descibed in detail in Section 3.1 of <a href="https://arxiv.org/abs/2312.03444" style="color: #2575fc; text-decoration: none; font-weight: bold;">this paper</a>). The resulting regression coefficients at each exercise date provide a stopping rule, which can be applied to the testing data to get true lower-bounds</li>
    <li><span style="color: #2575fc; font-weight: bold;">→</span> The <code style="background-color: #eef; padding: 2px 4px; border-radius: 3px;">LinearDualPricer</code> uses the signature of the training data to minimize over the familiy of linear signature martingales, by solving a corresponding linear program (described in Detail in Section 3.2 of <a href="https://arxiv.org/abs/2312.03444" style="color: #2575fc; text-decoration: none; font-weight: bold;">this paper</a>). The resulting coefficients yield a Doob martingale approximation, which for the testing data yields a true upper bound.</li>
</ul>
<p>By combining the two values, we receive confidence intervals for the true option price.</p>
</div>

<div style="background-color: #fff4e6; padding: 12px; border-radius: 5px; border-left: 4px solid #fd7e14; margin: 10px 0;">
<p>To solve the linear programm, one can optionally choose to use <a href="https://www.gurobi.com" style="color: #fd7e14; text-decoration: none; font-weight: bold;">Gurobi</a>, which requires a free licence, which is recommended especially for high-dimensional LPs, which occur when choosing large sample-sizes and/or high signature truncations levels. Alternatively, we use the free LP solvers from CVXPY</p>
</div>

In [ ]:
# ────────────────────────────────────────────────────────────────────────────────
# add root of repo and the “Linear signature optimal stopping” folder to PYTHONPATH
import sys, os
repo_root = os.path.abspath("..")  # /Users/.../Optimal_Stopping_with_signatures
ls_folder = os.path.join(repo_root, "Linear signature optimal stopping")
sys.path.extend([repo_root, ls_folder])

# now the module can be imported
from Linear_signature_optimal_stopping import LinearLongstaffSchwartzPricer, LinearDualPricer
# ────────────────────────────────────────────────────────────────────────────────

In [ ]:

#initialze the models
ls_pricer = LinearLongstaffSchwartzPricer(
        N1=N1,
        T=T_years,
        r=r,
        mode="American Option",
        ridge=10**(-9)
    )

dual_pricer = LinearDualPricer(
        N1=N1,
        N=N,
        T=T_years,
        r=r,
        LP_solver="CVXPY"
    )

The choice mode="American Option" indicates that the Longstaff-Schwartz recursion will only consider "in-the-money" paths, which was originally suggested by Longstaff & Schwartz, and is reasonable for non-negative payoffs. For general payoffs we can use mode = "Standard".

In [ ]:
#compute true lower bounds
lower_bound, lower_bound_std, ls_regression_models = ls_pricer.price(
        signatures_training,
        Payoff_training,
        signatures_testing,
        Payoff_testing
    )

In [ ]:
print(f"Linear Longstaff-Schwartz lower bound: {lower_bound} ± {lower_bound_std/np.sqrt(M2)}")

Similarly let us derive the upper bounds, but we will train the model only for $M= 5000$ paths to reduce computation time, and then compute true prices for all testing samples.

In [ ]:
M_dual = 5000
upper_bound, upper_bound_std, MG = dual_pricer.price(
        signatures_training[:M_dual],
        Payoff_training[:M_dual],
        dW_training[:M_dual,:,0],  # Select only the first component of the Brownian increments
        signatures_testing,
        Payoff_testing,
        dW_testing[:,:,0]  # Select only the first component of the Brownian increments
    )

In [ ]:
print(f"Linear Dual upper bound: {upper_bound} ± {upper_bound_std/np.sqrt(M2)}")
print(f"Pricing interval: {(float(lower_bound),float(upper_bound))}± {np.maximum(upper_bound_std,lower_bound_std)/np.sqrt(M2)} ")


# Improving the duality gap


Especially in rough regimes (here $H=0.1$), we observe a significant gap between lower and upper bounds, and in this section we present two ways to improve it. The first one still relies on linear signatures, but extends the basis as explained in in Section 4 of https://arxiv.org/abs/2312.03444.

## Part 1: Extending the linear basis

We consider a more involved basis by choosing the extended signature lift of $(t,X_t,\phi(X_t))$, and additionally add Laguerre polynomials of $(X_t,v_t)$. We can again use the SignatureComputer to compute this extended basis:

In [ ]:
sig_computer_extended = SignatureComputer(T, N, 3, "linear", signature_lift="payoff-and-polynomial-extended", poly_degree=3)

In [ ]:
signatures_extended_training = sig_computer_extended.compute_signature(
    S_training, vol_training, A_training, Payoff_training,
    dW_training, I_training, MM_training
)
signatures_extended_testing = sig_computer_extended.compute_signature(
    S_testing, vol_testing, A_testing, Payoff_testing,
    dW_testing, I_testing, MM_testing
)

Now we repeat the procedure for the extended basis:

In [ ]:

#compute true lower bounds for the new basis
lower_bound_extended, lower_bound_extended_std, ls_regression_models_extended = ls_pricer.price(
        signatures_extended_training,
        Payoff_training,
        signatures_extended_testing,
        Payoff_testing
    )
#Repeating the dual procedure for the new basis
upper_bound_extended, upper_bound_extended_std, MG_extended = dual_pricer.price(
        signatures_extended_training[:M_dual,:,:],
        Payoff_training[:M_dual,:],
        dW_training[:M_dual,:,0],  # select first component of Brownian increments
        signatures_extended_testing,
        Payoff_testing,
        dW_testing[:,:,0]  # select first component of Brownian increments
    )

In [ ]:
print(f"Improve pricing interval: {(float(lower_bound_extended),float(upper_bound_extended))}± {np.maximum(upper_bound_std,lower_bound_std)/np.sqrt(M2)} ")

## Part 2: Deep log-signature optimal stopping

 In forthcoming work about "American options in rough volatility models", we will focus on more non-linear apporaches to price American options. More precisely, we extend the primal and dual procecdure by replacing linear functionals of the signature by deep neural networks on the log-signature $\mathbb{L}=\mathrm{log}^\otimes(\mathbb{X})$. This transformed version of the signature still captures the relevant information about the past of the underlying process, but grows much slower as the signature it self with respect to the truncation. Then, in order to learn highly non-linear functionals, such as the integrand of the Doob martingale ("derivative of the Snell-envelope"), we apply deep feedforward neural networks $\theta$ on the log-signature. Of course, in both methods a optimization of the hyperparameters is required.

We proceed as before, but replace the linear signature by the log-signature

In [ ]:
sig_computer_log = SignatureComputer(T, N, 3, "log", signature_lift="polynomial-vol", poly_degree=3)

In [ ]:
log_signatures_training = sig_computer_log.compute_signature(
    S_training, vol_training, A_training, Payoff_training,
    dW_training[:,:,0], I_training, MM_training  # use first component
)
log_signatures_testing = sig_computer_log.compute_signature(
    S_testing, vol_testing, A_testing, Payoff_testing,
    dW_testing[:,:,0], I_testing, MM_testing  # use I_testing
)

In [ ]:
repo_root = os.path.abspath("..")  # /Users/.../Optimal_Stopping_with_signatures
ls_folder = os.path.join(repo_root, "Non linear signature optimal stopping")
sys.path.extend([repo_root, ls_folder])
from Deep_signatures_optimal_stopping import DeepLongstaffSchwartzPricer, DeepDualPricer

In [ ]:
log_signatures_training = sig_computer_log.compute_signature(
    S_training, vol_training, A_training, Payoff_training,
    dW_training[:,:,0], I_training, MM_training  # use first component
)
log_signatures_testing = sig_computer_log.compute_signature(
    S_testing, vol_testing, A_testing, Payoff_testing,
    dW_testing[:,:,0], I_testing, MM_testing  # use correct I_testing
)
print("shape of dW_training", dW_training.shape)
print("shape of dW_testing", dW_testing.shape)

The DeepLongstaffSchwartzPricer generalizes the LinearLongstaffSchwartzPrices, where the Ridge Regression at each exercise date is replace by learning the conditional expectations via neural networks. In the following initialization we build a network with $3$ hidden layers and $16$ neurons each, between each hidden layer we apply the activation function $\mathrm{tanh}(x)$. The remainding parameters are set to 'False'. (One can run the 'Hyperparameter_optimization_primal.py' file to optimize the choice of hyperparameters)

In [ ]:
ls_pricer = DeepLongstaffSchwartzPricer(
    N1=N1,
    T=T_years,
    r=r,
    mode="American Option",
    layers=3,
    nodes=16,
    activation_function='tanh',
    batch_normalization=False,
    regularizer=0.0,  # This is correct as float
    dropout=False,
    layer_normalization=False
)

dual_pricer = DeepDualPricer(
    N1=N1,
    N=N,
    T=T_years,
    r=r,
    layers=3,
    nodes=16,
    activation_function='relu',
    batch_normalization=False,
    regularizer=0.0,  # ERROR: should be float, not boolean
    dropout=False,
    attention_layer=False,
    layer_normalization=False
)
# LS pricer call is correct
lower_bound_deep, lower_bound_deep_std, ls_regression_models = ls_pricer.price(
    log_signatures_training,
    Payoff_training,
    log_signatures_testing,
    Payoff_testing,
    M_val=0,
    batch=2**8,
    epochs=15,
    learning_rate=0.001
)

# Dual pricer call is correct
y0, upper_bound_deep, upper_bound_deep_std, dual_model, dual_rule_model = dual_pricer.price(
    log_signatures_training,
    Payoff_training,
    dW_training[:,:,0],  # use only first component of Brownian increments
    log_signatures_testing,
    Payoff_testing,
    dW_testing[:,:,0],  # use only first component of Brownian increments
    M_val=int(0.9*M),
    batch=2**8,
    epochs=15,
    learning_rate=0.01
)

Similarly for the dual problem, we consider the same network but use the $relu(x)$ activation instead.

In [ ]:
# Consistent parameter usage for validation set size
M_val_percentage = 0.9

The Deep Longstaff Schwartz uses $15$ epochs for at the last exercise date, and then one epochs at the remainding ones by initiliazing smartly. The learning rate for the Stochastic Gradient Descent is choosen as $0.001$, and we use batch sizes of $2^8$.

In [ ]:
print(f"Deep Longstaff-Schwartz lower bound: {lower_bound_deep} ± {lower_bound_deep_std/np.sqrt(M2)}")

In [ ]:
print(f"Deep Dual upper bound: {upper_bound_deep} ± {upper_bound_deep_std/np.sqrt(M2)}")
print(f"Pricing interval: {(lower_bound_deep,upper_bound_deep)}± {np.maximum(upper_bound_deep_std,lower_bound_deep_std)/np.sqrt(M2)} ")

In [ ]:
repo_root = os.path.abspath("..")  # /Users/.../Optimal_Stopping_with_signatures
ls_folder = os.path.join(repo_root, "Non linear signature optimal stopping")
sys.path.extend([repo_root, ls_folder])
from Deep_kernel_signature_optimal_stopping import DeepKernelLongstaffSchwartzPricer, DeepKernelDualPricer


In [ ]:
# Cell 1: Define the RFF feature computation functions with corrected implementation

def compute_rff_kernel_features(signatures, N1, rff_dim=128, gamma=1.0):
    """
    Compute Random Fourier Features for lower bound pricer (list format)
    
    Args:
        signatures: Signature data with shape [M, T_steps, feature_dim]
        N1: Number of exercise dates
        rff_dim: Dimension of random features (default: 128)
        gamma: RBF kernel bandwidth parameter (default: 1.0)
    
    Returns:
        List of tensors with shape [M, rff_dim*2, 1] for each exercise date
    """
    M, T_steps, feature_dim = signatures.shape
    
    # Calculate indices based on actual data dimensions
    actual_steps = T_steps - 1
    subindex = [min(int((j+1)*actual_steps/N1), actual_steps) for j in range(N1)]
    
    print(f"Signature data has {T_steps} time points")
    print(f"Using exercise indices: {subindex}")
    
    # Create list to hold RFF features for each exercise date
    rff_features_list = []
    
    # For each exercise date
    for t in range(len(subindex)):
        idx = min(subindex[t], T_steps-1)
        X_t = signatures[:, idx, :]
        
        # Generate random projection matrix for RBF kernel approximation
        np.random.seed(42 + t)  # Different seed for each exercise date
        W = np.random.normal(0, np.sqrt(2*gamma), (feature_dim, rff_dim))
        
        # Compute RFF: [cos(Wx), sin(Wx)]
        projection = X_t @ W
        rff_features = np.column_stack([
            np.cos(projection),
            np.sin(projection)
        ]) * np.sqrt(1/rff_dim)
        
        # Reshape to match expected format: [M, rff_dim*2, 1]
        rff_features = rff_features.reshape(M, rff_dim*2, 1)
        
        rff_features_list.append(rff_features)
    
    return rff_features_list

def compute_rff_kernel_features_dual(signatures, N, N1, rff_dim=128, gamma=1.0):
    """
    Compute Random Fourier Features for dual pricer (3D tensor format)
    
    Args:
        signatures: Signature data with shape [M, T_steps, feature_dim]
        N: Number of time steps in the discretization
        N1: Number of exercise dates
        rff_dim: Dimension of random features (default: 128)
        gamma: RBF kernel bandwidth parameter (default: 1.0)
    
    Returns:
        Tensor with shape [M, features, time] for all time points
    """
    M, T_steps, feature_dim = signatures.shape
    
    # Calculate indices proportional to exercise dates
    # We need to map our exercise indices to the full discretization grid
    actual_steps = min(T_steps - 1, N)
    all_indices = np.minimum(np.array([int(t * T_steps / (N+1)) for t in range(N+1)]), T_steps-1)
    
    print(f"Using exercise indices for dual pricer: {all_indices[:5]}...{all_indices[-5:]}")
    
    # Generate random projection matrix once
    np.random.seed(42)
    W = np.random.normal(0, np.sqrt(2*gamma), (feature_dim, rff_dim))
    
    # Extract all required signature data at once
    X_all = signatures[:, all_indices, :]  # Shape: [M, N+1, feature_dim]
    
    # Reshape for batch matrix multiplication
    X_reshaped = X_all.reshape(-1, feature_dim)  # Shape: [M*(N+1), feature_dim]
    
    # Compute all projections at once
    projections = X_reshaped @ W  # Shape: [M*(N+1), rff_dim]
    
    # Compute RFF features
    cos_features = np.cos(projections)
    sin_features = np.sin(projections)
    rff_features = np.column_stack([cos_features, sin_features]) * np.sqrt(1/rff_dim)
    
    # Reshape back to original dimensions
    full_rff = rff_features.reshape(M, N+1, rff_dim*2)
    
    # Transpose to match expected format: [M, features, time]
    return np.transpose(full_rff, (0, 2, 1))

In [ ]:
# Cell 2: Calculate and use RFF features for pricing with corrected implementation

# Calculate both sets of features
rff_dim = 64
print("Computing kernel features for lower bound...")
kernel_training = compute_rff_kernel_features(log_signatures_training, N1, rff_dim=rff_dim)
kernel_testing = compute_rff_kernel_features(log_signatures_testing, N1, rff_dim=rff_dim)

# IMPORTANT: For the dual approach, we need to generate features for N1 steps
print("Computing kernel features for upper bound...")
kernel_training_dual = compute_rff_kernel_features_dual(log_signatures_training, N1, N1, rff_dim=rff_dim)
kernel_testing_dual = compute_rff_kernel_features_dual(log_signatures_testing, N1, N1, rff_dim=rff_dim)
print("Initializing kernel-based lower bound...")
# 1. LOWER BOUND calculation - this works correctly
kernel_pricer = DeepKernelLongstaffSchwartzPricer(
    N1=N1,
    T=T_years,
    r=r,
    L=rff_dim*2,
    mode="American Option",
    layers=3,
    nodes=32,
    activation_function='relu',
    batch_normalization=True,
    regularizer=0.001,
    dropout=False,
    layer_normalization=True
)

print("Computing kernel-based lower bound...")
lower_bound_kernel, lower_bound_kernel_std, kernel_models = kernel_pricer.price(
    kernel_training,
    kernel_testing,
    Payoff_training,
    Payoff_testing,
    batch=2**8,
    epochs=15,
    learning_rate=0.0005
)

# 2. UPPER BOUND calculation - use direct payoff, no need to expand
print("Initializing kernel-based upper bound...")
kernel_dual_pricer = DeepKernelDualPricer(
    N1=N1,
    N=N1,  # Using N1 instead of N=252 here is the key change
    T=T_years,
    r=r,
    layers=4,
    nodes=32,
    activation_function='relu',
    batch_normalization=True,
    regularizer=0.001,
    dropout=True,
    attention_layer=False,
    layer_normalization=True,
    mode_dim="1-dim"
)
print("Computing kernel-based upper bound...")
try:
    y0_kernel, upper_bound_kernel, upper_bound_kernel_std, kernel_model, kernel_rule_model = kernel_dual_pricer.price(
        kernel_training_dual,
        Payoff_training,
        dW_training[:,:,0],
        kernel_testing_dual,
        Payoff_testing,      
        dW_testing[:,:,0],
        M_val=int(0.9*M),
        batch=2**8,
        epochs=15,
        learning_rate=0.0005
    )
    
    # Report results
    print(f"Deep Kernel Longstaff-Schwartz lower bound: {lower_bound_kernel} ± {lower_bound_kernel_std/np.sqrt(M2)}")
    print(f"Deep Kernel Dual upper bound: {upper_bound_kernel} ± {upper_bound_kernel_std/np.sqrt(M2)}")
    print(f"Kernel-based pricing interval: [{lower_bound_kernel}, {upper_bound_kernel}] ± {np.maximum(upper_bound_kernel_std, lower_bound_kernel_std)/np.sqrt(M2)}")
except Exception as e:
    print(f"Error in dual pricer: {e}")
    print(f"Deep Kernel Longstaff-Schwartz lower bound: {lower_bound_kernel} ± {lower_bound_kernel_std/np.sqrt(M2)}")
    print("Upper bound calculation failed - using only lower bound")

We once again stress that the parameters for the the discretization (here $J=120$), the sample size (here $M=10^{15}$), and the signature trunaction level (here $K=3$) are not choosen big enough to get narrow gaps, but we can still already observe an improvement.

## Step 5: Contextualizing Theoretical Price in USD
Convert the normalized model price bounds into USD per share and per contract, and print actionable trading recommendations.

In [ ]:
# Convert the normalized price bounds to actual USD values
actual_stock_price = X0  # USD per share
actual_strike = strike  # USD per share

# Include all four methods in the list
methods = [
    "Linear Signature", 
    "Extended Linear Signature", 
    "Deep Log-Signature",
    "Deep Kernel Method"
]
print(lower_bound_kernel, upper_bound_kernel, lower_bound_kernel_std, upper_bound_kernel_std)

# Actual Bond Premium stored as variable premium
actual_premium = model_inputs["market_premium"]

# Collect all price bounds
lower_bounds = [lower_bound, lower_bound_extended, lower_bound_deep, lower_bound_kernel]
upper_bounds = [upper_bound, upper_bound_extended, upper_bound_deep, upper_bound_kernel]
stds = [lower_bound_std, lower_bound_extended_std, lower_bound_deep_std, lower_bound_kernel_std]

# Create a table of results with duality gap
import pandas as pd
from IPython.display import display, HTML

results = []
for i, method in enumerate(methods):
    usd_lower = float(lower_bounds[i])
    
    # Handle the case where upper bound might not be available for kernel method
    if i == 3 and 'upper_bound_kernel' not in locals():
        usd_upper = float('nan')  # Use NaN if upper bound isn't available
    else:
        usd_upper = float(upper_bounds[i]) 
    
    usd_std = float(stds[i]) / np.sqrt(M2)
    
    # Calculate duality gap only if upper bound exists
    if not np.isnan(usd_upper):
        duality_gap = usd_upper - usd_lower
        gap_percent = duality_gap / usd_lower * 100
    else:
        duality_gap = float('nan')
        gap_percent = float('nan')
    
    # Determine if premium is within bounds
    if not np.isnan(usd_upper):
        premium_status = "Within bounds" if usd_lower <= actual_premium <= usd_upper else "Outside bounds"
    else:
        premium_status = "Compared to lower bound only"
    
    results.append({
        "Method": method,
        "Lower Bound (USD)": f"${usd_lower:.2f}",
        "Upper Bound (USD)": f"${usd_upper:.2f}" if not np.isnan(usd_upper) else "N/A",
        "Std Error (USD)": f"${usd_std:.2f}",
        "Duality Gap (USD)": f"${duality_gap:.2f}" if not np.isnan(duality_gap) else "N/A",
        "Gap (%)": f"{gap_percent:.2f}%" if not np.isnan(gap_percent) else "N/A",
        "Market Premium": f"${actual_premium:.2f}",
        "Premium Status": premium_status
    })

results_df = pd.DataFrame(results)
display(HTML(results_df.to_html(index=False)))

# Print text comparison as well
print(f"\nMarket Premium: ${actual_premium:.2f}")
print("\nComparison with calculated bounds:")

for i, method in enumerate(methods):
    lower = float(lower_bounds[i])
    upper = float(upper_bounds[i]) if i < 3 or 'upper_bound_kernel' in locals() else float('nan')
    
    print(f"{method}:")
    print(f"  Lower Bound: ${lower:.2f}  {'(underprices)' if lower < actual_premium else '(overprices)'}")
    
    if not np.isnan(upper):
        print(f"  Upper Bound: ${upper:.2f}  {'(underprices)' if upper < actual_premium else '(overprices)'}")
        print(f"  Duality Gap: ${upper - lower:.2f} ({(upper - lower)/lower*100:.2f}%)")
        print(f"  Premium within bounds: {'Yes' if lower <= actual_premium <= upper else 'No'}")
    
    print()

## Problem with Rough Volatility Simulation

# This highlights a limitation of our current rough volatility approach. Because the Hurst exponent we estimated is around 0.7, the model assumes a relatively smooth (less rough) path and ends up over-pricing option premia. Rough volatility frameworks are designed to capture very irregular (highly rough) volatility dynamics typically corresponding to more volatile stocks so when applied to smoother data (H = 0.7) they tend to inflate theoretical premiums compared to actual market prices.

## Option Trading Interpretation

Now let's interpret these results from a trading perspective. We'll evaluate the fair price range for an American put option contract (which typically represents 100 shares).

In [ ]:
# Now we can calculate the fair price range for a standard options contract
# Cell under “# For a standard options contract (100 shares)”
shares_per_contract = 100
contract_lower   = float(lower_bound_deep) * actual_stock_price
contract_upper   = float(upper_bound_deep) * actual_stock_price
contract_midpoint = (contract_lower + contract_upper) / 2

print(f"American Put Option Contract Analysis (for {shares_per_contract} shares)")
print(f"=====================================================================")
print(f"Stock Price: ${actual_stock_price:.2f}")
print(f"Strike Price: ${actual_strike:.2f}")
print(f"Time to Maturity: {T} days")
print(f"Interest Rate: {r*100:.2f}%")
print(f"Rough Volatility Parameters: H={H}, η={eta}, ρ={rho}, ξ₀={xi}")
print(f"=====================================================================")
print(f"Fair Price Range: ${contract_lower:.2f} to ${contract_upper:.2f} per contract")
print(f"Midpoint Price: ${contract_midpoint:.2f}")
print(f"=====================================================================")

# Trading recommendations based on market prices
hypothetical_market_prices = [contract_lower * 0.8, contract_midpoint, contract_upper * 1.2]
labels = ["Below Fair Value", "At Fair Value", "Above Fair Value"]

print("Trading Recommendations:")
for price, label in zip(hypothetical_market_prices, labels):
    print(f"\nIf market price is ${price:.2f} ({label}):")
    
    if price < contract_lower:
        print("→ BUY: Market price is below fair value range")
        print(f"→ Expected edge: ${(contract_lower - price):.2f} to ${(contract_upper - price):.2f} per contract")
        print("→ Consider buying puts for protection or speculative profit")
    elif price > contract_upper:
        print("→ SELL: Market price is above fair value range")
        print(f"→ Expected edge: ${price - contract_upper:.2f} to ${price - contract_lower:.2f} per contract")
        print("→ Consider writing puts, potentially as part of a spread strategy to limit risk")
    else:
        print("→ NEUTRAL: Market price is within fair value range")
        position = (price - contract_lower) / (contract_upper - contract_lower)
        print(f"→ Price is positioned {position:.0%} through the fair value range")
        if position < 0.4:
            print("→ Slight bias toward buying")
        elif position > 0.6:
            print("→ Slight bias toward selling")
        else:
            print("→ No clear edge for buying or selling")

## Risk Management Considerations

When trading American put options in a rough volatility environment, several risk management considerations are important:

In [ ]:
# Get actual market premium from the data
market_premium = actual_premium 
percent_itm = max(0, (actual_strike - actual_stock_price) / actual_strike)

moneyness = actual_stock_price / actual_strike
# Corrected time value calculation for a put option
intrinsic_value = max(0, actual_strike - actual_stock_price)
time_value = market_premium - intrinsic_value

rounded_hurst = [round(item, 2) for item in Hurst_list]

print("Risk Management Considerations:")
print("=====================================================================")
print(f"Moneyness: {moneyness:.2f} ({percent_itm:.0%} in-the-money)")
print(f"Intrinsic Value: ${intrinsic_value:.2f} per share")
print(f"Time Value: ${time_value:.2f} per share")
print(f"Total Premium: ${market_premium:.2f} per share")
print(f"Uncertainty Range: ${(contract_upper - contract_lower):.2f} per contract")
print("\nRecommended Risk Management Strategies:")
print("---------------------------------------------------------------------")
print("1. Position Sizing: Limit exposure to <5% of portfolio per trade")
print("2. Early Exercise Consideration: Monitor optimal stopping boundaries")
print("3. Hedging: Consider delta and vega hedging for larger positions")
print("4. Model Risk: Be aware model assumes H={rounded_hurst}, may differ from market")

# Additional practical advice
print("\nPractical Implementation:")
print("---------------------------------------------------------------------")
if moneyness < 0.95:
    print("→ Deep ITM option: Consider early exercise if dividend yield > interest rate")
    print("→ Watch for significant changes in volatility that could shift optimal exercise boundary")
elif moneyness > 1.05:
    print("→ OTM option: Early exercise unlikely, trade like European option")
    print("→ Primary value is in insurance against downside moves")
else:
    print("→ ATM option: Maximum gamma/vega exposure")
    print("→ Most sensitive to changes in volatility and rough volatility parameters")
    print("→ Actively monitor for optimal early exercise conditions near expiration")

print("\nNote: This model incorporates rough volatility effects H=({rounded_hurst}) which\n")
print("traditional models like Black-Scholes miss. This can be particularly")
print("important for managing risk in volatile market conditions.")

## Environment Configuration for TensorFlow
<div style="background-color: #E8F4FD; padding: 12px; border-radius: 5px; border-left: 5px solid #2196F3; margin: 10px 0;">
This section handles Python path configuration and environment setup for TensorFlow compatibility. We need to:

1. Add the necessary directories to the Python path to make modules available for import
2. Suppress TensorFlow warnings which can be distracting in a notebook environment
3. Set the TensorFlow CPP log level to minimize non-essential messages

This is particularly important for the deep learning pricing methods we'll use later in the notebook.
</div>